In [0]:
from pyspark.sql.functions import * 

In [0]:
# Azure Event Hub Configuration
event_hub_namespace = "hospital-analytics-eh-namespace.servicebus.windows.net"
event_hub_name="hospital-analytics-eh"  
event_hub_conn_str = dbutils.secrets.get(scope = "hospitalanalyticsvaultscope", key = "eventhub-connection")

In [0]:
# connecting to event hub
kafka_options = {
    'kafka.bootstrap.servers': f"{event_hub_namespace}:9093",
    'subscribe': event_hub_name,
    'kafka.security.protocol': 'SASL_SSL',
    'kafka.sasl.mechanism': 'PLAIN',
    'kafka.sasl.jaas.config': f'kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username="$ConnectionString" password="{event_hub_conn_str}";',
    'startingOffsets': 'latest',
    'failOnDataLoss': 'false'
}

In [0]:
# reading stream from event hub
raw_df = (spark.readStream
          .format("kafka")
          .options(**kafka_options)
          .load()
          )
#Cast data to json (converting binary data in raw json string)
json_df = raw_df.selectExpr("CAST(value AS STRING) as raw_json")

#ADLS configuration 
spark.conf.set(
  "fs.azure.account.key.hospitalstorageac.dfs.core.windows.net",
  dbutils.secrets.get(scope = "hospitalanalyticsvaultscope", key = "storage-connection")
)

bronze_path = "abfss://bronze@hospitalstorageac.dfs.core.windows.net/patient_flow"

#Write stream to bronze
(
    json_df
    .writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", "dbfs:/mnt/bronze/_checkpoints/patient_flow")
    .start(bronze_path)
)

In [0]:
display(spark.read.format("delta").load(bronze_path))